A Python script that generates a GIF from a numpy npy volume.

In [4]:
import numpy as np
import napari
import os
from PIL import Image

# small # 1.2 # 2.0
data_paths = [
    # '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_ori.npy',
    # '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_conv5.npy',
    #'/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_median5.npy',
    #'/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_wiener5.npy',
    #'../data/battery/4_battery_clean_crops/saves/4-means.npy',
    '../data/z-volumes/r2gaus/180_avg/180_avg_r2gaus_enh.npy'
    # '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_bm4d.npy',
    # '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_n2v.npy',
    # '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_liao.npy',
    # '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_smallhand/volume_756_3d_ca5_ci1_samG_denTT_lrc1_rand0_50000_20000.npy',
]

# large # 1.0 # 1.1
# data_paths = [
#     '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_largehand/hand_volume_756_ori.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_largehand/hand_volume_756_conv3.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_largehand/hand_volume_756_bm4d.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_largehand/hand_volume_756_n2v.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_largehand/hand_volume_756_liao.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/output_volumes_largehand/volume_756_3d_ca3_ci1_samG_denTT_lrc1_rand0_50000_70000.npy',
# ]

# forearm-1 # 0.6 # 0.6
# data_paths = [
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-1-volume-756-08/forearm-1-volume-756-08.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-1-volume-756-08/forearm-1-volume-756-08_cov4.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-1-volume-756-08/forearm-1-volume-756-08_bm4d.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-1-volume-756-08/forearm-1-volume-756-08_n2v.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-1-volume-756-08/forearm-1-volume-756-08_ca4_ci1_samG_denTT_infT_lrc1_rand0_50000_100000.npy',
# ]

# forearm-2 # 0.6 # 0.6
# data_paths = [
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-2-volume-756-08/forearm-2-volume-756-08.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-2-volume-756-08/forearm-2-volume-756-08_cov4.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-2-volume-756-08/forearm-2-volume-756-08_bm4d.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-2-volume-756-08/forearm-2-volume-756-08_n2v.npy',
#     '/Users/kaminyou/Desktop/pa_output_volumes/forearm-2-volume-756-08/forearm-2-volume-756-08_ca4_ci1_samG_denTT_infT_lrc1_rand0_70000_80000.npy',
# ]

# set up background volume
#background = '../data/battery/4_battery_clean_crops/exp_1/r2gaus/volume_180_enh.npy'
background = '../data/z-volumes/saxnerf/180_no_avg/mini/180_no_avg_saxnerf_cropped.npy'
background_volume = np.load(background)

start = None #None # 72 # 90
end = 120
zoom = 5.0
for data_path in data_paths:
    #basename = os.path.basename(data_path).rsplit('.', 1)[0]
    basename = 'out'

    if start is not None:
        image_3d = np.load(data_path)[start:end]
    else:
        image_3d = np.load(data_path)

    viewer = napari.Viewer(ndisplay=3, show=True)
    viewer.add_image(background_volume, name=basename, colormap='twilight_shifted')
    #viewer.add_labels(image_3d, name=basename, opacity=0.9)

    #viewer.camera.angles = (60, 0, 0)  # Rotations: z=30°, x=45°
    viewer.camera.zoom = zoom  # Adjust zoom level if needed
    #viewer.camera.angles = (10, -35, 140)

    pillow_images = []
    for i in range(0, 360, 4):
        viewer.camera.angles = (90, 20, i)
        x = viewer.screenshot(canvas_only=True)
        image = Image.fromarray(x[:, :, :3])
        original_width, original_height = image.size

        # Calculate the new dimensions (scaling by 0.5)
        new_width = int(original_width * 0.5)
        new_height = int(original_height * 0.5)

        # Resize the image while maintaining the aspect ratio
        image = image.resize((new_width, new_height))
        pillow_images.append(image)
        # image.save("/Users/kaminyou/Desktop/x.png")

    if start is None:
        #output_name = f'/Users/kaminyou/Desktop/Meeting_NII/videos/{basename}.gif'
        #output_name = f'../data/battery/4_battery_clean_crops/saves/{basename}.gif'
        output_name = f'../data/z-volumes/saxnerf/180_no_avg/mini/{basename}.gif'
    else:
        #output_name = f'/Users/kaminyou/Desktop/Meeting_NII/videos/{basename}_{start}-{end}.gif'
        output_name = f'../data/battery/4_battery_clean_crops/saves/{basename}_{start}-{end}.gif'

    pillow_images[0].save(
        # "/Users/kaminyou/Desktop/animated.gif",
        output_name,
        save_all=True,
        append_images=pillow_images[1:],
        duration=150,  # Duration between frames in milliseconds
        loop=0         # Loop forever (set to 1 for single loop)
    )

    viewer.close()
    # napari.run()

Now we create a slideshow of each 2D slice and save it as GIF.

In [52]:
import numpy as np
import napari
import os
from PIL import Image
from tqdm import tqdm

output_folder = '../data/battery/4_battery_clean_crops/saves/slices/'

background = '../data/battery/4_battery_clean_crops/exp_1/saxnerf/volume_180.npy'
#foreground = '../data/battery/4_battery_clean_crops/saves/4-means.npy'

background_volume = np.load(background)
#foreground_volume = np.load(foreground)

viewer = napari.Viewer()
viewer.add_image(background_volume, name='3D Image', colormap='twilight_shifted')
#viewer.add_labels(foreground_volume, name='3D Labels', opacity=0.9)

scaling_factor = 0.5

# Function to scale down an image
def scale_down_image(image, factor):
    pil_image = Image.fromarray(image)
    new_size = (
        int(pil_image.width * factor),
        int(pil_image.height * factor),
    )
    return np.array(pil_image.resize(new_size, Image.Resampling.LANCZOS))  # Use LANCZOS for high-quality downsampling

# Loop through slices and save images
for i in tqdm(range(background_volume.shape[0])):
    viewer.dims.set_point(0, i)  # Set the current slice
    screenshot = viewer.screenshot(canvas_only=True)
    resized_screenshot = scale_down_image(screenshot, scaling_factor)
    screenshot_path = os.path.join(output_folder, f'slice_{i:03d}.png')
    napari.utils.io.imsave(screenshot_path, resized_screenshot)
print(f'Saved.')

100%|██████████| 256/256 [00:39<00:00,  6.50it/s]

Saved.


In [53]:
import imageio.v2 as imageio

# Load saved images and create a GIF
images = []
for i in tqdm(range(background_volume.shape[0])):
    img_path = os.path.join(output_folder, f'slice_{i:03d}.png')
    images.append(imageio.imread(img_path))

# Save the images as an animated GIF
output_gif = '../data/battery/4_battery_clean_crops/saves/3d_slices_animation.gif'
imageio.mimsave(output_gif, images, duration=0.1, loop=0)  # duration = time per frame in seconds

print(f'GIF saved as {output_gif}')

100%|██████████| 256/256 [00:03<00:00, 79.17it/s]


GIF saved as ../data/battery/4_battery_clean_crops/saves/3d_slices_animation.gif
